In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Bipolar_disorder/GSE93114'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and MicroRNA expression data from lymphoblastoid cell lines from patients with bipolar disorder"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: bipolar disorder'], 1: ['response phenotype, alda scale: excellent responders', 'response phenotype, alda scale: non-responders'], 2: ['cell type: lymphoblastoid cell line']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Determine if the dataset contains gene expression data.
if "Gene and MicroRNA expression data" in "!Series_title".lower():
    is_gene_available = True

# Step 2: Check for the availability of Bipolar_disorder, age, and gender data.
sample_characteristics_dict = {
    0: ['disease state: bipolar disorder'],
    1: ['response phenotype, alda scale: excellent responders', 'response phenotype, alda scale: non-responders'],
    2: ['cell type: lymphoblastoid cell line']
}

# Identify the keys for each variable
trait_row = 0 if 0 in sample_characteristics_dict and len(set(sample_characteristics_dict[0])) > 1 else None
age_row = None  # No age data found
gender_row = None  # No gender data found

# Step 3: Define conversion functions
def convert_trait(value):
    trait_mapping = {
        "disease state: bipolar disorder": 1,
        "disease state: control": 0
    }
    return trait_mapping.get(value.split(": ")[-1], None)

def convert_age(value):
    try:
        # Extracting age if listed as age: value
        return float(value.split(": ")[-1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender_mapping = {
        "gender: male": 1,
        "gender: female": 0
    }
    return gender_mapping.get(value.split(": ")[-1], None)

# Step 4: Save cohort information
save_cohort_info('GSE93114', './preprocessed/Bipolar_disorder/cohort_info.json', is_gene_available, trait_row is not None)

# Step 5: Extract clinical features if available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Bipolar_disorder', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Bipolar_disorder/trait_data/GSE93114.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
